In [1]:
import cv2
import numpy as np

from collections import deque
from timeit import default_timer as timer

# Catch frame from webcam
camera = cv2.VideoCapture(r'C:\file location..')

# Define variables for hight and width shape of the frames
HEIGH, WIDTH = 300, 400   

# Scale to decrease the frame size
SCALE = 1

# Define objects boundaries size
MIN_OBJECT_AREA = 1000
MAX_OBJECT_AREA = 10000

# Define the background
last_frame = np.zeros((HEIGH, WIDTH,3) , np.uint8)
background = np.zeros((HEIGH, WIDTH,3) , np.uint8)

# Initializing deque object for center points of the detected object
points = deque(maxlen=50)

# Initialize TRACKING variables
tracking_on = False

# Restart timer for FPS
fps_start = timer()    

# Define tracking frames counter
counter_frames_tracking = 0

# Define reading frames counter
counter_frames_reading = 0

# Define processing frames counter
counter_frames_processing = 0

# Define prediction frames counter
counter_frames_prediction = 0

# Define timer to check the tracking
tracking_check = 15

# Variable store the system status of tracking or not tracking
tracking_on = False

# Define the tresh hold of the masks
DIFF_TRESH_HOLD = 20  # Should be low
MASK_TRESH_HOLD = 100 # Should be high

# Increasing FPS counter
counter_fps = 0

# Define FPS Variable
FPS = 0

# Define tracker dictionary
tracker_dict = { 'csrt': cv2.TrackerCSRT_create,
                 'kcf' : cv2.TrackerKCF_create,
                 'boosting' : cv2.legacy.TrackerMOSSE_create(),
                 'mil': cv2.TrackerMIL_create,
                 'tld': cv2.legacy.TrackerTLD_create(),
                 'medianflow': cv2.legacy.TrackerMedianFlow_create(),
                 'mosse':cv2.legacy.TrackerMOSSE_create()}

# Initialize our tracker after the object
tracker = tracker_dict['csrt']()

########################################################################################################################
    
# Function return 3-Dimension frame
def expands_dimensions(frame):
    
    new_image = np.zeros((HEIGH, WIDTH, 3), np.uint8)
    new_image[:, :, 0] = frame
    new_image[:, :, 1] = frame
    new_image[:, :, 2] = frame
    
    return new_image

# Convert frame from rgb to gray
def gray_frame(frame_rgb):
    
    # Converting captured frame to GRAY by OpenCV function    
    gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    frame_gray = np.zeros(frame_rgb.shape, np.uint8)
    frame_gray[:,:,0] = gray_frame
    frame_gray[:,:,1] = gray_frame
    frame_gray[:,:,2] = gray_frame
    
    return frame_gray

# Function define the mask
def mask(frame_rgb, background):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  

    # Converting captured frame to GRAY by OpenCV function        
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    background = cv2.GaussianBlur(background, (25, 25), 0)    
    
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, background)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask = cv2.threshold(abs_diff, 20, 255, cv2.THRESH_BINARY)
    
    # Dilates the object in the frame
    dilated_mask = cv2.dilate(mask, None, iterations = 5) 
    
    return dilated_mask

def mask_tracking(frame_RGB, last_frame):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_RGB, cv2.COLOR_RGB2GRAY)
    
    # Converting captured frame to GRAY by OpenCV function        
    last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
     
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, last_frame)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, abs_diff_mask = cv2.threshold(abs_diff, DIFF_TRESH_HOLD, 255, cv2.THRESH_BINARY)

    # Expend mask dimension to 3 dimension
    mask_frame = expands_dimensions(abs_diff_mask)        
    
    return mask_frame

# Function manage the frames reader variables like efps etc'
def reader_manger(FPS, fps_start, counter_fps, tracking_on):
       
    # Variable says if keep reading frame or quit
    quit = False

    # Stopping the timer for FPS
    fps_stop = timer()

    # Print FPS every 1 second
    if 1.0 <= fps_stop - fps_start:

        # Define FPS
        FPS = counter_fps

        # Reset FPS counter
        counter_fps = 0

        # Restart timer for FPS
        fps_start = timer()       

    # Function waits for key to be pressed    
    key = cv2.waitKey(10) % 256

    # If 'n' is pressed, we catchs the frame and define it as the background
    if key == ord('n'):
        tracking_on = False

    # If 'q' key is pressed then quit from app
    if key == ord('q'):
        quit = True   

    return FPS, fps_start, counter_fps, tracking_on, quit

# Function display the frames on the screen in one window
def display_windows(frame, tracking_mask, frame_mask):  
        
    # Copy frame to work with deffrent variable
    tracking_mask = cv2.resize(tracking_mask, (WIDTH, HEIGH))
        
    # Copy frame to work with deffrent variable
    frame_mask = cv2.resize(frame_mask, (WIDTH, HEIGH))
        
    # Expend mask dimension to 3 dimension
    frame_mask = expands_dimensions(frame_mask)       
    
    # Create left window    
    main_window = np.hstack((frame, tracking_mask, frame_mask))

    # Plotting all the frames in one window
    cv2.imshow("Main_Window", main_window)     

# Function create 3 frames from the frame we read
def preproccess_frames(frame, last_frame):
    
    # Define small sizes
    heigh, width = int(HEIGH/SCALE), int(WIDTH/SCALE)
    
    # Resize the main frame to (WIDTH, HEIGH) shape
    frame = cv2.resize(frame, (WIDTH, HEIGH))
        
    # Copy frame to work with deffrent variable
    frame_rgb = cv2.resize(frame, (width, heigh))
    
    # Return mask frame 
    frame_mask = mask(frame_rgb, last_frame)
    
    # Return mask for tracking
    tracking_mask = mask_tracking(frame_rgb, last_frame)
    
    # Define last frame
    last_frame = frame_rgb.copy()
    
    return frame, frame_rgb, tracking_mask, frame_mask, last_frame

# Function manage the detection and return status and coordinates
def detection_manager(frame, frame_rgb, frame_mask, counter_frames_prediction):
    
    # Increase prediction counter
    counter_frames_prediction += 1    
    
    # Function return array of all contours we found
    contours, _ = cv2.findContours(frame_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sorted the contours and define the larger first
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Scan the contours list
    for contour in contours:

        # Return square area of the given contour
        contour_area = cv2.contourArea(contour)

        # Find contours between MIN_OBJECT_AREA to MAX_OBJECT_AREA
        if contour_area < MAX_OBJECT_AREA:
            if MIN_OBJECT_AREA < contour_area:
                
                # Get an approximate rectangle coordinates
                (x_min, y_min, box_width, box_height) = cv2.boundingRect(contour)
            
                # bounding_boxes contain x1, y1, x2, y2, coordinates and not width and heigh
                (x_min, y_min, box_width, box_height) = x_min*SCALE, y_min*SCALE, box_width*SCALE, box_height*SCALE

                # Drawing rectangle on the frame
                frame = cv2.rectangle(frame, (x_min, y_min), (x_min +box_width, y_min +box_height), (0, 255, 0), 2)
                
                # Store the rectangle coordinates around the object
                rectangle = np.array([x_min, y_min, box_width, box_height])
                
                # Start tracking after the object
                tracker.init(frame_rgb, rectangle)

                # Define tracking status
                tracking_on = True
                
                return frame, frame_rgb, tracking_on, counter_frames_prediction
            else:
                # Contour is a sorted list so all the rest items irrelevant
                break
    
    # No object to follow after found
    tracking_on = False
    
    return frame, frame_rgb, tracking_on, counter_frames_prediction

# Function manage the tracking and return the status
def tracking_manager(frame, tracking_mask, frame_mask, tracker, counter_frames_reading):     

    # Set the default status
    tracking_on = False

    # Get the bounding box from the frame
    (success, contour_box) = tracker.update(frame)

    # Keep tracking after the object
    if success:

        # Get the coordinates of the rectangle around the object
        (x, y, w, h) = [int(a) for a in contour_box]

        # Check if coordinates is in the frame boundaries
        if 0 < x and x+w < WIDTH and 0 < y and y+h < HEIGH:   

            # Cut the fragment from the mask frame
            cut_fragment_mask = tracking_mask[int(y/SCALE):int((y+h)/SCALE), int(x/SCALE):int((x+w)/SCALE)]

            # Checking if tracking is still running after the object or not
            if((counter_frames_reading%tracking_check == 0 and np.sum(cut_fragment_mask) == 0)):      

                # Set tracking status OFF
                tracking_on = False
            else:
                # Set tracking status ON
                tracking_on = True                       

                # Drawing bounding box on the current BGR frame        
                cv2.rectangle(frame, (x,y), (x+w,y+h), (100,255,0), 2)

                # Putting text with label on the current BGR frame
                cv2.putText(frame, "label", (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)    
    
    return frame, tracking_mask, tracker, tracking_on

In [2]:
# Loop reading frame by frame and processing them
while True:
    
    # Increasing FPS counter
    counter_fps += 1
    
    # Capturing frames one-by-one from camera
    ret, frame = camera.read()

    # If the frame was not retrieved then we break the loop
    if not ret or frame is None:
        break
     
    # Increasing frames counter
    counter_frames_reading += 1

    # Function return 3 diffrent kind of frames
    frame, frame_rgb, tracking_mask, frame_mask, last_frame = preproccess_frames(frame, last_frame)
    
    
    # Start Proccessing state only if is not quite frames
    if 0 < np.sum(frame_mask):

        # Start Tracking state
        if tracking_on == True:

            # Increase tracking counter
            counter_frames_tracking += 1
            
            # Function manage the the tracking part  
            frame, tracking_mask, tracker, tracking_on = tracking_manager(frame, tracking_mask, frame_mask, tracker, counter_frames_reading)

        # Start Detection state
        else:

            # Increase tracking counter
            counter_frames_processing += 1

            # Function find countors around the objects and return drawn frame
            frame, frame_rgb, tracking_on, counter_frames_prediction = detection_manager(frame, frame_rgb, frame_mask, counter_frames_prediction)
        

    # Display all frames in one window
    display_windows(frame, tracking_mask, frame_mask)

    # Function manage the frames reader variables
    FPS, fps_start, counter_fps, tracking_on, quit = reader_manger(FPS, fps_start, counter_fps, tracking_on)

    # If quit is true so we stop read frames
    if quit == True:
        break

# Releasing camera
camera.release()

# Destroying all opened OpenCV windows
cv2.destroyAllWindows()      